In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import cv2
import numpy as np
import pandas as pd
from scipy.optimize import shgo, linprog
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dataclasses
from scipy.spatial.transform import Rotation

from slam_backend.homegeneous_projection import HomVectors, HomMatrix

In [ ]:
u = HomVectors(x=1, y=2, z=3)
v = HomVectors(x=4, y=5, z=6)
w = u.concatenate(v)

In [ ]:
w._data

In [ ]:
Path().resolve()

In [ ]:
@dataclasses.dataclass(frozen=True)
class CameraParameters:
    camera_matrix: np.ndarray
    distortion_coefficients: np.ndarray
    image_shape: np.array
    rotation_vectors: np.ndarray
    translation_vectors: np.ndarray

hm01b0_params = CameraParameters(
    camera_matrix=np.load(Path("res/camera_calibration/HM01B0/properties/camera_matrix.npy")),
    distortion_coefficients=np.load(Path("res/camera_calibration/HM01B0/properties/distortion_coefficients.npy")),
    image_shape=np.load(Path("res/camera_calibration/HM01B0/properties/image_shape.npy")),
    rotation_vectors=np.load(Path("res/camera_calibration/HM01B0/properties/rotation_vectors.npy")),
    translation_vectors=np.load(Path("res/camera_calibration/HM01B0/properties/translation_vectors.npy"))
)

def load_and_rectify_image(path: Path, camera_parameters: CameraParameters, crop=False) -> np.ndarray:
    image = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)

    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(
        camera_parameters.camera_matrix,
        camera_parameters.distortion_coefficients,
        (camera_parameters.image_shape[1], camera_parameters.image_shape[0]),
        1,
        (camera_parameters.image_shape[1], camera_parameters.image_shape[0]),   
    )
    mapx, mapy = cv2.initUndistortRectifyMap(
        camera_parameters.camera_matrix,
        camera_parameters.distortion_coefficients,
        None,
        newcameramtx,
        (camera_parameters.image_shape[1],camera_parameters.image_shape[0]), 
        5
    )
    dst = cv2.remap(image, mapx, mapy, cv2.INTER_LINEAR)
 
    if crop:
        x, y, w, h = roi
        dst = dst[y:y+h, x:x+w]
    return dst

In [ ]:
df_state = pd.read_csv("C:/Users/filip/Documents/atomichighfive/crazyflie-slam/output/image-streamer/20240808-023209/state.csv")
base_path = Path("C:/Users/filip/Documents/atomichighfive/crazyflie-slam/")

display(df_state.head(3))

# Raw state estimates

In [ ]:
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Add a scatter3d trace for the moving point
scatter = fig.add_traces(
    [
        go.Scatter3d(
            x=[df_state.iloc[0]["position_x"]],  # Initial x position
            y=[df_state.iloc[0]["position_y"]],  # Initial y position
            z=[df_state.iloc[0]["position_z"]],  # Initial z position
            mode='markers',
            marker=dict(
                size=4,
                color='red'
            ),
            name='Moving Point'
        ),
        go.Scatter3d(
            x=[df_state.iloc[0]["position_x"], df_state.iloc[0]["position_x"] + df_state.iloc[0]["acceleration_x"]],
            y=[df_state.iloc[0]["position_y"], df_state.iloc[0]["position_y"] + df_state.iloc[0]["acceleration_y"]],
            z=[df_state.iloc[0]["position_z"], df_state.iloc[0]["position_z"] + df_state.iloc[0]["acceleration_z"]],
            mode='lines',
            line=dict(
                color='green',
                width=4
            ),
            name='Acceleration Vector'
        ),
    ]
)

# Set the layout
fig.update_layout(
    scene=dict(
        xaxis=dict(range=[df_state.position_x.min(), df_state.position_x.max()], autorange=False),
        yaxis=dict(range=[df_state.position_y.min(), df_state.position_y.max()], autorange=False),
        zaxis=dict(range=[df_state.position_z.min(), df_state.position_z.max()], autorange=False),
        aspectratio=dict(x=1, y=1, z=1),
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5)
        )
    ),
    title='3D Point Animation'
)

# Define the animation frames
frames = []
for i in range(len(df_state)):
    frame = go.Frame(
        data=[
            go.Scatter3d(
                x=[df_state.iloc[i]["position_x"]],
                y=[df_state.iloc[i]["position_y"]],
                z=[df_state.iloc[i]["position_z"]],
                mode='markers',
                marker=dict(
                    size=4,
                    color='red'
                ),
                name='position'
            ),
            go.Scatter3d(
                x=[df_state.iloc[i]["position_x"], df_state.iloc[i]["position_x"] + df_state["acceleration_x"].ewm(alpha=2/3).mean().iloc[i]],
                y=[df_state.iloc[i]["position_y"], df_state.iloc[i]["position_y"] + df_state["acceleration_y"].ewm(alpha=2/3).mean().iloc[i]],
                z=[df_state.iloc[i]["position_z"], df_state.iloc[i]["position_z"] + df_state["acceleration_z"].ewm(alpha=2/3).mean().iloc[i]],
                mode='lines',
                line=dict(
                    color='green',
                    width=4
                ),
                name='Acceleration Vector'
            ),
        ]
    )
    frames.append(frame)

# Add the frames to the figure
fig.frames = frames

# Set the animation settings
animation_settings = dict(
    frame=dict(duration=100, redraw=True),
    fromcurrent=True,
    transition=dict(duration=500, easing='quadratic-in-out')
)

# Add the animation buttons
fig.update_layout(
    updatemenus=[
        dict(
            type='buttons',
            buttons=[
                dict(label='Play',
                     method='animate',
                     args=[None, animation_settings]),
                dict(label='Pause',
                     method='animate',
                     args=[[None], animation_settings])
            ],
            showactive=False,
            x=0.1,
            y=0,
            xanchor='center',
            yanchor='bottom'
        )
    ]
)

# Show the figure
fig.show()

# Image processing

In [ ]:
def difference_of_gaussians(image: np.ndarray, sigma1: float, sigma2: float) -> np.ndarray:
    blurred1 = cv2.GaussianBlur(image, (0, 0), sigma1)
    blurred2 = cv2.GaussianBlur(image, (0, 0), sigma2)
    return blurred1 - blurred2

def detect_edge_points(image: np.ndarray, edge_mask_pixels=5, mode='points') -> np.ndarray:
    assert mode in ('points', 'image'), "mode must be either 'points' or 'image'"
    x = image[edge_mask_pixels:-edge_mask_pixels, edge_mask_pixels:-edge_mask_pixels]
    x = cv2.morphologyEx(x, cv2.MORPH_GRADIENT, np.ones((3, 3), np.uint8))
    x = cv2.threshold(x, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    x = cv2.morphologyEx(x, cv2.MORPH_OPEN, np.ones((2, 2), np.uint8))
    x = np.concatenate(
        [
            np.zeros((x.shape[0]+edge_mask_pixels*2, edge_mask_pixels), dtype=x.dtype),
            np.concatenate(
                [np.zeros((edge_mask_pixels, x.shape[1]), dtype=x.dtype), x, np.zeros((edge_mask_pixels, x.shape[1]), dtype=x.dtype)],
                axis = 0
            ),
            np.zeros((x.shape[0]+edge_mask_pixels*2, edge_mask_pixels), dtype=x.dtype),
        ],
        axis=1
    )
    if mode == 'image':
        return x
    
    points = np.argwhere(x == 1)
    return points

def detect_sift_features(image: np.ndarray, edge_mask_pixels=5) -> np.ndarray:
    x = image[edge_mask_pixels:-edge_mask_pixels, edge_mask_pixels:-edge_mask_pixels]
    x = np.concatenate(
        [
            np.zeros((x.shape[0]+edge_mask_pixels*2, edge_mask_pixels), dtype=x.dtype),
            np.concatenate(
                [np.zeros((edge_mask_pixels, x.shape[1]), dtype=x.dtype), x, np.zeros((edge_mask_pixels, x.shape[1]), dtype=x.dtype)],
                axis = 0
            ),
            np.zeros((x.shape[0]+edge_mask_pixels*2, edge_mask_pixels), dtype=x.dtype),
        ],
        axis=1
    )
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(x, None)
    keypoints = cv2.KeyPoint_convert(keypoints)
    keypoints = keypoints[:, ::-1]
    return keypoints, descriptors

def match_points(
        points1: np.ndarray,
        descriptors1: np.ndarray,
        points2: np.ndarray,
        descriptors2: np.ndarray,
) -> np.ndarray:
    matcher = cv2.BFMatcher(
        normType=cv2.NORM_L2,
        crossCheck=True
    )
    matches = matcher.knnMatch(descriptors1, descriptors2, k=1, compactResult=True)

    #print(len(points1), len(points2), len(matches))

    matched_points1, matched_points2 = map(
        np.array,
        zip(*[[points1[match[0].queryIdx], points2[match[0].trainIdx]] for match in matches if len(match)])
    )

    # Drop the 10% longest distance matches, because sequential images are expected to have matches close in distance
    sum_of_squared_distances = np.sum((matched_points1 - matched_points2) ** 2, axis=1)
    matched_points1 = matched_points1[np.argsort(sum_of_squared_distances)]
    matched_points2 = matched_points2[np.argsort(sum_of_squared_distances)]

    matched_points1 = matched_points1[:int(len(matched_points1) * 0.9)]
    matched_points2 = matched_points2[:int(len(matched_points2) * 0.9)]

    return matched_points1, matched_points2
    

    


In [ ]:
img1 = load_and_rectify_image(
    df_state["output_file"].iloc[0],
    camera_parameters=hm01b0_params
)
img2 = load_and_rectify_image(
    df_state["output_file"].iloc[1],
    camera_parameters=hm01b0_params
)

keypoints1, descriptors1 = detect_sift_features(img1)
keypoints2, descriptors2 = detect_sift_features(img2)

res = match_points(
    keypoints1,
    descriptors1,
    keypoints2,
    descriptors2
)

display(res)

In [ ]:
img = load_and_rectify_image(df_state["output_file"].iloc[32], camera_parameters=hm01b0_params)
points = detect_edge_points(img)

In [ ]:
img = load_and_rectify_image(df_state["output_file"].iloc[32], camera_parameters=hm01b0_params)
points, descriptors = detect_sift_features(img)

display(points, descriptors)

In [ ]:
# Create the figure
fig = go.Figure()

# Add two subplots, one for the original image and one for the edge-detected image
fig = make_subplots(rows=1, cols=2)

# Add the original image to the first subplot
fig.add_trace(go.Heatmap(z=load_and_rectify_image(df_state["output_file"].iloc[0], camera_parameters=hm01b0_params)), row=1, col=1)

# Add the edge-detected image to the second subplot
edge_points, edge_descriptors = detect_sift_features(
    load_and_rectify_image(
        df_state["output_file"].iloc[0],
        camera_parameters=hm01b0_params
    )
)
next_edge_points, next_edge_descriptors = detect_sift_features(
    load_and_rectify_image(
        df_state["output_file"].iloc[1],
        camera_parameters=hm01b0_params
    )
)

matched_edge_points, matched_next_edge_points = match_points(
    edge_points,
    edge_descriptors,
    next_edge_points,
    next_edge_descriptors
)

line_segments_x = sum(
    [[u[1], v[1], None] for u, v in zip(matched_edge_points, matched_next_edge_points)],
    []
)
line_segments_y = sum(
    [[u[0], v[0], None] for u, v in zip(matched_edge_points, matched_next_edge_points)],
    []
)

for points_col in range(1, 3):
    fig.add_trace(
        go.Scatter(
            x=edge_points[:, 1],
            y=edge_points[:, 0],
            mode='markers',
            marker=dict(
                size=4,
                color='red'
            ),
            name=f'Points {points_col}'
        ),
        row=1,
        col=points_col
    )

fig.add_trace(
    go.Scatter(
        x=next_edge_points[:, 1],
        y=next_edge_points[:, 0],
        mode='markers',
        marker=dict(
            size=4,
            color='blue'
        ),
        name='Next Points'
    ),
    row=1,
    col=2
)

fig.add_trace(
    go.Scatter(
        x=line_segments_x,
        y=line_segments_y,
        mode='lines',
        name='Matches',
        line=go.scatter.Line(color='black', width=1)
    ),
    row=1,
    col=2
)

# Set the layout
fig.update_layout(
    title='Image Animation',
    updatemenus=[
        dict(
            type='buttons',
            buttons=[
                dict(label='Play',
                     method='animate',
                     args=[None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True, 'transition': {'duration': 0}}]),
                dict(label='Pause',
                     method='animate',
                     args=[[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate', 'transition': {'duration': 0}}])
            ],
            showactive=False,
            x=0.1,
            y=0,
            xanchor='center',
            yanchor='bottom'
        )
    ]
)

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Frame:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# Define the animation frames
frames = []
for i in range(len(df_state)-1):
    edge_points, edge_descriptors = next_edge_points, next_edge_descriptors
    
    next_edge_points, next_edge_descriptors = detect_sift_features(
        load_and_rectify_image(
            df_state["output_file"].iloc[i+1],
            camera_parameters=hm01b0_params
        )
    )

    matched_edge_points, matched_next_edge_points = match_points(
        edge_points,
        edge_descriptors,
        next_edge_points,
        next_edge_descriptors
    )

    line_segments_x = sum(
        [[u[1], v[1], None] for u, v in zip(matched_edge_points, matched_next_edge_points)],
        []
    )
    line_segments_y = sum(
        [[u[0], v[0], None] for u, v in zip(matched_edge_points, matched_next_edge_points)],
        []
    )

    frame = go.Frame(
        data=[
            go.Heatmap(
                z=load_and_rectify_image(
                    df_state["output_file"].iloc[i],
                    camera_parameters=hm01b0_params
                )
            ),
            go.Scatter(
                x=edge_points[:, 1],
                y=edge_points[:, 0],
                mode='markers',
                marker=dict(
                    size=4,
                    color='red'
                ),
                name='Points 0'
            ),
            go.Scatter(
                x=edge_points[:, 1],
                y=edge_points[:, 0],
                mode='markers',
                marker=dict(
                    size=4,
                    color='red'
                ),
                name='Points 1'
            ),
            go.Scatter(
                x=next_edge_points[:, 1],
                y=next_edge_points[:, 0],
                mode='markers',
                marker=dict(
                    size=4,
                    color='blue'
                ),
                name='Next Points'
            ),
            go.Scatter(
                x=line_segments_x,
                y=line_segments_y,
                mode='lines',
                name='Matches'
             )
        ],
        layout=go.Layout(
            title=f'Frame {i+1}'
        ),
        name=str(i)
    )
    frames.append(frame)
    slider_step = {
        "args": [
            [i],
            {"frame": {"duration": 0, "redraw": True},
            "mode": "immediate",
            "transition": {"duration": 0}}
        ],
        "label": str(i),
        "method": "animate"
    }
    sliders_dict["steps"].append(slider_step)

# Add the frames to the figure
fig.frames = frames

# Show the figure
fig['layout']['yaxis']['autorange'] = "reversed"
fig['layout']['yaxis2']['autorange'] = "reversed"
#fig.update_yaxes(
#    scaleanchor = "x",
#    scaleratio = 1,
#    row=1, col=1
#)
#fig.update_yaxes(
#    scaleanchor = "x2",
#    scaleratio = 1,
#    row=1, col=2
#)
#fig.update_traces(showscale=False, col=1, row=1)
fig['layout']['sliders'] = [sliders_dict]

fig.show()


In [ ]:
def estimate_transformation(image_1, image_2):
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(image_1, None)
    kp2, des2 = orb.detectAndCompute(image_2, None)

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)

    matches = sorted(matches, key = lambda x:x.distance)

    good = matches[:50]

    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

    #M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    F, _mask = cv2.findFundamentalMat(src_pts, dst_pts, cv2.FM_RANSAC, 5.0)

    M = np.concatenate(
        [
            np.concatenate(
                [F, np.array([[0, 0, 0]])],
                axis=0
            ),
            np.array([[0, 0, 0, 1]]).T
        ],
        axis=1
    )

    T = M[3, :3]

    return M, T

In [ ]:
hm01b0_params.rotation_vectors.shape

In [ ]:
hm01b0_params.camera_matrix

In [ ]:
hm01b0_params.rotation_vectors.shape